In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations_with_replacement

In [7]:
#append relevant file paths
new_path = pathl('.')
parent = new_path.resolve().parent
sys.path.append(str(parent))

In [8]:
import os, sys
import numpy as np
from pathlib import Path as pathl
from pdf_parser import pipeline

In [9]:
import spacy
import pickle
nlp = spacy.load('en_core_web_sm')
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~\n'

In [10]:
current_dir = os.walk(sys.path[-1] + '/Data/downloaded_files')
files = []
for file in current_dir:
    files.append(file[-1])
files = sum(files, [])

In [11]:
data_path = parent / 'Data/downloaded_files'
def get_text(filename):
    'Return text from a filename'
    pdf_file = data_path / filename
    text_dict = pipeline(filepath = str(pdf_file))
    text = list(text_dict.values())
    text = sum(text, [])
    text = [sentence.strip() for sentence in text]
    text = ' '.join(text)
    return text

def lemmatizer(text):
    'Lemmatizes text'
    doc = nlp.pipe(text)
    lemmatized = []
    for sentence in doc:
        sent = []
        for word in sentence:
            if str(word) in punctuation:
                continue
            lemma = word.lemma_.strip() 
            sent.append(lemma)
        
        lemmatized.append(' '.join(sent))
    return lemmatized

In [36]:
vectorizer = TfidfVectorizer(min_df=1)
corpus = []
counter = 0
for file in files:
    if file[-3:] == 'pdf':
        try:
            text = get_text(file)
            text = text.split('.')
            lemmatized = lemmatizer(text)
            corpus.append(' '.join(lemmatized))
        except:
            print(f'Failed: {counter}')
            counter+=1

Failed: 0
Failed: 1
Failed: 2
Failed: 3
Failed: 4
Failed: 5


Failed: 6


In [37]:
corpus = [i for i in corpus if len(i) > 1]

In [38]:
model = vectorizer.fit_transform(corpus)
dense = model.todense()

In [91]:
similarity = [cosine_similarity(model_1, model_2) for model_1, model_2 in list(combinations_with_replacement(dense, 2))]
similarity = [similarity[i][0][0] for i in range(len(similarity))]

In [93]:
similarity = np.array(similarity)
similarity = np.round(similarity, 2)
similarity_of_pdfs = list(zip(list(combinations_with_replacement(range(0,len(corpus)), 2)), similarity))
len(similarity_of_pdfs)

2080

In [98]:
import pandas as pd

In [103]:
df_list = []
for (text1, text2), similarity in similarity_of_pdfs:
    df_list.append((text1, text2, similarity))

In [107]:
df = pd.DataFrame(df_list, columns=['text_1', 'text_2', 'similarity'])